In [1]:
from experiments._1_one_user_learn_neighbours.try_some_users import *
from experiments.utils import *
from experiments.datasets import *

/home/pablo/.virtualenvs/tesiscomp/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/pablo/.virtualenvs/tesiscomp/local/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Switching to API Credentials #5


In [2]:
uid = 228252737
s = open_session()
user_ld = s.query(User).get(uid)
# X_train, X_test, y_train, y_test = load_or_create_dataset(uid)
X_train, X_test, y_train, y_test = load_or_create_dataframe(uid)

In [3]:
neighbours = get_neighbourhood(uid)

In [4]:
len(user_ld.timeline)

2871

In [5]:
from sklearn.ensemble import RandomForestClassifier
clf_class = RandomForestClassifier

# weights for class balancing
w1 = sum(y_train)/len(y_train)
w0 = 1 - w1
sample_weights = np.array([w0 if x==0 else w1 for x in y_train])

print("Training %s" % clf_class.__name__)
clf = clf_class()     
clf.fit(X_train, y_train, sample_weight=sample_weights)

y_true, y_pred = y_train, clf.predict(X_train)

print("Detailed classification report:\n")
print("Scores on training set.\n")
print(classification_report(y_true, y_pred))

y_true, y_pred = y_test, clf.predict(X_test)
print("Scores on test set.\n")
print(classification_report(y_true, y_pred))

Training RandomForestClassifier
Detailed classification report:

Scores on training set.

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00    165770
        1.0       1.00      0.84      0.91      2019

avg / total       1.00      1.00      1.00    167789

Scores on test set.

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00     71058
        1.0       0.97      0.81      0.88       852

avg / total       1.00      1.00      1.00     71910



Analicemos las probabilidades para tener una idea de la ¿perplejidad? del modelo

In [35]:
maxprobs = []
probas = clf.predict_proba(X_test)
for r in probas:
    maxprobs.append(max(r))

print sum(maxprobs)

print probas.sum()

71881.8269124
71910.0


In [ ]:

clf.predict_proba(X_test)

Este usuario muestra un recall mucho más bajo que los demás.
Esta pérdida de calidad puede deberse a errores forzosos causados por ambigüedades en el dataset de entrenamiento.
Diremos que hay ambigüedad para un vector de features $v$ si $(v,0)$ y $(v,1)$ ocurren ambos una cantidad significativa de veces.

In [6]:
miss_clf_counts, details = count_doomed_samples(X_train, y_train)

In [7]:
details[:30]

[([u'Linyera'], {0.0: 1265, 1.0: 27}),
 ([u'D\xe9bora '], {0.0: 1241, 1.0: 27}),
 ([u'Daniel Ventura'], {0.0: 749, 1.0: 8}),
 ([u'Carlos Russo'], {0.0: 297, 1.0: 7}),
 ([u'Laprogre Quespera'], {0.0: 781, 1.0: 7}),
 ([u'infobae'], {0.0: 1832, 1.0: 6}),
 ([u'EmmaDLRS\u270c'], {0.0: 584, 1.0: 6}),
 ([u'Aldo Ulises Jarma'], {0.0: 376, 1.0: 6}),
 ([u'Pol\xedtica Argentina'], {0.0: 421, 1.0: 6}),
 ([u'Mauricio Macri'], {0.0: 52, 1.0: 5}),
 ([u'Todo Negativo'], {0.0: 463, 1.0: 5}),
 ([u'Super Secretario'], {0.0: 270, 1.0: 5}),
 ([u'  Helena'], {0.0: 260, 1.0: 5}),
 ([u'gem'], {0.0: 849, 1.0: 4}),
 ([u'\u049aari\u0438oOlgoDesakatado'], {0.0: 1476, 1.0: 4}),
 ([u'Wakje'], {0.0: 1219, 1.0: 4}),
 ([u'Silvio \xae'], {0.0: 1639, 1.0: 3}),
 ([u'6-7-8'], {0.0: 605, 1.0: 3}),
 ([u'Alex Freyre'], {0.0: 415, 1.0: 3}),
 ([u'#\u5de8\u5927\u30ea\u30d0\u30fc\u30d7\u30ec\u30fc\u30c8'],
  {0.0: 1235, 1.0: 3}),
 ([u'laura amado'], {0.0: 77, 1.0: 3}),
 ([u'pedro garcia'], {0.0: 214, 1.0: 3}),
 ([u'guillermo dib

In [8]:
miss_clf_counts

defaultdict(float, {0: 7.5, 1: 301.5})

Las ambigüedades parecen venir mayormente de retweets a usuarios con las siguientes características:

* La mayoría de sus tweets no son retweeteados por LD.
* pocos seguidores en el vecindario de LD

In [23]:
error_sources = [d for d in details if d[1][1.0] >= 5]

Veamos si realmente estos usuarios tienen tan pocos seguidores en el entorno de LD

In [24]:
unames = [x[0][0] for x in error_sources]
users = {u.username: u for u in neighbours if u.username in unames}

In [25]:
g = load_nx_graph()

In [26]:
s = open_session()

In [27]:
followers = []
nids = [u.id for u in neighbours]
for un in unames: # importance order
    fs = get_followers(users[un], s, g)
    fs = [f for f in fs if f.id in nids]
    followers.append((un, fs))

In [28]:
follower_counts = [(un, len(fs)) for (un, fs) in followers]
follower_counts

[(u'Linyera', 26),
 (u'D\xe9bora ', 1),
 (u'Daniel Ventura', 16),
 (u'Carlos Russo', 14),
 (u'Laprogre Quespera', 1),
 (u'infobae', 2),
 (u'EmmaDLRS\u270c', 1),
 (u'Aldo Ulises Jarma', 27),
 (u'Pol\xedtica Argentina', 46),
 (u'Mauricio Macri', 15),
 (u'Todo Negativo', 60),
 (u'Super Secretario', 138),
 (u'  Helena', 3)]

Parecen tener más seguidores de lo esperado en el entorno. Veamos si estos seguidores retweetean sus tweets o no.

In [29]:
tl_counts = defaultdict(int)
for t in user_ld.timeline:
    tl_counts[t.author_id] += 1

In [30]:
len(tl_counts)

1209

In [18]:
len(user_ld.timeline)

2871

In [32]:
def contar_retweets(user, followers):
    count = 0
    for f in followers:
        rts = [t for t in f.timeline if t.author_id == user.id]
        count += len(rts)
    return count  

In [33]:
rt_counts = {}
for un in unames:
    user = users[un]
    rt_counts[un] = contar_retweets(user, fs)

In [34]:
rt_counts

{u'  Helena': 0,
 u'Aldo Ulises Jarma': 0,
 u'Carlos Russo': 0,
 u'Daniel Ventura': 0,
 u'D\xe9bora ': 0,
 u'EmmaDLRS\u270c': 0,
 u'Laprogre Quespera': 0,
 u'Linyera': 0,
 u'Mauricio Macri': 0,
 u'Pol\xedtica Argentina': 0,
 u'Super Secretario': 0,
 u'Todo Negativo': 1,
 u'infobae': 0}

In [ ]:
len(twids)

In [ ]:
tws = s.query(Tweet).filter(Tweet.id.in_(twids))
[t.text for t in tws[:10]]